# Islands

In the Virgin Islands / Puerto Rico area we are running into several places where it would be helpful to have a consistent 30 meter raster depiction of islands.

This is an attempt to make that.

We are using the CUSP shorelines for an indicator, and they have a pretty detailed shoreline, so we are going to try to match to that.

Because I am doing a layer select, this code doesn't work unless is is run with an open map in Pro

In [82]:
import os
import arcpy

In [83]:
import time
start = time.time()

In [84]:
# define spatial reference and workspaces
sr= arcpy.SpatialReference(5070)
#SourceWorkspace= 
OutWorkspace= r"D:\SE_Blueprint_2023\Islands\Isl.gdb"

In [85]:
# define final indicator outputs
Out = r"D:\SE_Blueprint_2023\Islands\IslandsR.tif"
Out2 = r"D:\SE_Blueprint_2023\Islands\Islands3Size.tif"
OutL = r"D:\SE_Blueprint_2023\Islands\IslandLarge.tif"
OutM = r"D:\SE_Blueprint_2023\Islands\IslandMedium.tif"
OutS = r"D:\SE_Blueprint_2023\Islands\IslandSmall.tif"
OutMar = r"D:\SE_Blueprint_2023\Islands\Marine.tif"
OutSMar = r"D:\SE_Blueprint_2023\Islands\IslandSmallOrMarine.tif"

In [86]:
# define sub-indicator outputs to help with user support


In [87]:
# define rasters used for cell size, extent, and snapping
Rextent= r"D:\SE_Blueprint_2023\1_VIPR_Extent\VIPR_Extent_v6.tif"

In [88]:
# define inputs

# since the catchments don't cover the small islands, we need to make a layer for those islands
# I'm using the CUSP shoreline data, since it seems to have more recent shorelines
shore =r"F:\GIS_DATA\LanduseLandcover\CUSP_Shoreline\Southeast_Caribbean\Southeast_Caribbean.shp"


### Start Analysis

In [89]:
# Set the workspace where I want the output to go
arcpy.env.workspace = OutWorkspace

In [90]:
# Print the current workspace to make sure I'm in the right spot
print(arcpy.env.workspace)

D:\SE_Blueprint_2023\Islands\Isl.gdb


### make islands

In [91]:
# make a copy of the shoreline dataset that only covers VIPR, reproject to our projection
with arcpy.EnvManager(outputCoordinateSystem=sr, extent=Rextent):
    arcpy.management.CopyFeatures(shore, "shore1", '', None, None, None)

In [92]:
# add and calculate a field to disolve on
arcpy.management.CalculateField("shore1", "dis", "1", "PYTHON3", "", "LONG")

<Result 'D:\\SE_Blueprint_2023\\Islands\\Isl.gdb\\shore1'>

In [93]:
# dissolve the shorelines to see where they don't connect
arcpy.management.Dissolve("shore1", "shore1Dis", "dis", None, "SINGLE_PART", "DISSOLVE_LINES")

<Result 'D:\\SE_Blueprint_2023\\Islands\\Isl.gdb\\shore1Dis'>

### Do some weird stuff to get the PR polygon to close

The CUSP data doesn't close the polygons on the big island of Puerto Rico, so I have to do some werid stuff to get that one.


In [94]:
# make a copy of the dissolved shorelines so we'll have one to edit and one to save
arcpy.management.CopyFeatures("shore1Dis", "shore1DisEdit", '', None, None, None)

<Result 'D:\\SE_Blueprint_2023\\Islands\\Isl.gdb\\shore1DisEdit'>

In [95]:
# Use the integrate tool to snap nearby nodes together, to collapse the lines on the big island of PR
# we need this for the big island of PR, but we are doing it for everything here
with arcpy.EnvManager(XYTolerance="8 Meters"):
    arcpy.analysis.PairwiseIntegrate("shore1DisEdit", None)

In [96]:
# convert modified shore lines to polygons
with arcpy.EnvManager(outputCoordinateSystem=sr, extent=Rextent):
    arcpy.management.FeatureToPolygon("shore1DisEdit", "Prtest", None, "NO_ATTRIBUTES", None)

In [97]:
# add a field so we can dissolve
arcpy.management.CalculateField("PrTest", "dissolve", "1", "PYTHON3", '', "SHORT", "NO_ENFORCE_DOMAINS")

<Result 'D:\\SE_Blueprint_2023\\Islands\\Isl.gdb\\PrTest'>

In [98]:
# dissolve islands so that waterbodies on islands are included as a part of the island
arcpy.management.Dissolve("PrTest", "PrTestDis", "dissolve", None, "SINGLE_PART", "DISSOLVE_LINES")

<Result 'D:\\SE_Blueprint_2023\\Islands\\Isl.gdb\\PrTestDis'>

In [99]:
# select out the big island of PR
arcpy.analysis.Select("PrTestDis", "PR", "Shape_Area > 8000000000")

<Result 'D:\\SE_Blueprint_2023\\Islands\\Isl.gdb\\PR'>

### Make the rest of the islands

Now I need to go back and make the rest of the islands

In [100]:
# convert shore lines to polygons
with arcpy.EnvManager(outputCoordinateSystem=sr, extent=Rextent):
    arcpy.management.FeatureToPolygon("shore1Dis", "islandtest", None, "NO_ATTRIBUTES", None)

In [101]:
# add a field so we can dissolve
arcpy.management.CalculateField("islandTest", "dissolve", "1", "PYTHON3", '', "SHORT", "NO_ENFORCE_DOMAINS")

<Result 'D:\\SE_Blueprint_2023\\Islands\\Isl.gdb\\islandTest'>

In [102]:
# dissolve islands so that waterbodies on islands are included as a part of the island
arcpy.management.Dissolve("islandTest", "islandTestdis", "dissolve", None, "SINGLE_PART", "DISSOLVE_LINES")

<Result 'D:\\SE_Blueprint_2023\\Islands\\Isl.gdb\\islandTestdis'>

### Make the rest of the islands

Now I need to keep only the PR island and the other islands

In [103]:
# remove most island areas that overlap with the big PR island I just made

# first I do a select

with arcpy.EnvManager(outputCoordinateSystem=sr, extent=Rextent):
    arcpy.management.SelectLayerByLocation("islandtestdis", "INTERSECT", "PR", "", "NEW_SELECTION", "INVERT")
    # add back in the south part of Culebra
    arcpy.management.SelectLayerByAttribute("islandtestdis", "ADD_TO_SELECTION", "Shape_Area > 2610000 And Shape_Area < 2620000", None)


In [104]:
# Now I make a new layer based on that select

with arcpy.EnvManager(outputCoordinateSystem=sr, extent=Rextent):
    arcpy.management.CopyFeatures("islandtestdis", "islandNoIntersect", '', None, None, None)


In [105]:
# Now I clear that selection
arcpy.management.SelectLayerByAttribute("islandtestdis", "CLEAR_SELECTION", "", None)


<Result 'islandtestdis_Layer26'>

In [106]:
# remove remaining islands smaller than 900 square meters
#arcpy.analysis.Select("islandNoIntersect", "islands", "Shape_Area > 900")

In [107]:
# add an id field
arcpy.management.CalculateField("islandNoIntersect", "IslandID", "!OBJECTID!", "PYTHON3", '', "SHORT", "NO_ENFORCE_DOMAINS")

<Result 'D:\\SE_Blueprint_2023\\Islands\\Isl.gdb\\islandNoIntersect'>

In [108]:
# merge Islands with the PR big island
with arcpy.EnvManager(outputCoordinateSystem=sr, extent=Rextent):
    arcpy.analysis.Union(["islandNoIntersect","PR"], "islands", "ALL", None, "GAPS")

In [109]:
# create a field to make uniqu ids for islands
arcpy.management.CalculateField("islands", "UniqID", "!OBJECTID!", "PYTHON3", '', "SHORT", "NO_ENFORCE_DOMAINS")

<Result 'D:\\SE_Blueprint_2023\\Islands\\Isl.gdb\\islands'>

In [110]:
# create a field to convert to raster
arcpy.management.CalculateField("islands", "raster", "1", "PYTHON3", '', "SHORT", "NO_ENFORCE_DOMAINS")

<Result 'D:\\SE_Blueprint_2023\\Islands\\Isl.gdb\\islands'>

In [111]:
# convert polygons to raster
with arcpy.EnvManager(outputCoordinateSystem=sr, extent=Rextent, snapRaster=Rextent, cellSize=Rextent):
    arcpy.conversion.FeatureToRaster("islands", "raster", "islandsR", 30)

In [112]:
# convert lines to raster
with arcpy.EnvManager(outputCoordinateSystem=sr, extent=Rextent, snapRaster=Rextent, cellSize=Rextent):
    arcpy.conversion.FeatureToRaster("shore1", "dis", "shore1R", 30)

In [113]:
# combine 
out_raster = arcpy.sa.CellStatistics("islandsR;shore1R", "MAXIMUM", "DATA", "SINGLE_BAND", 90, "AUTO_DETECT"); out_raster.save("islandsR2")

In [114]:
# clump to give unique values to individual islands
out_raster = arcpy.sa.RegionGroup("islandsR2", "FOUR", "WITHIN", "ADD_LINK", None); out_raster.save("islands2rg")

In [115]:
# reclassify to make 3 island size classes to distinguish the big island of PR from medium sized islands
# from small islands. Also make not islands a 1, so we can use that for marine
with arcpy.EnvManager(outputCoordinateSystem=sr, extent=Rextent, snapRaster=Rextent, cellSize=Rextent):
    out_raster = arcpy.sa.Reclassify("islands2rg", "Count", "NODATA 1;0 30000 2;30000 8000000 3; 8000000 10000000 4", "DATA"); out_raster.save("island3size")

In [116]:
# make a raster for the big island
out_raster= arcpy.sa.Con("island3size", 1, 0, "Value = 4"); out_raster.save(OutL)

In [117]:
# make a raster for the medium islands
out_raster= arcpy.sa.Con("island3size", 1, 0, "Value = 3"); out_raster.save(OutM)

In [118]:
# make a raster for the smaill islands
out_raster= arcpy.sa.Con("island3size", 1, 0, "Value = 2"); out_raster.save(OutS)

In [119]:
# make a raster for the smaill island plus marine
out_raster = arcpy.sa.Con("island3size", 1, 0, "Value IN (1, 2)"); out_raster.save(OutSMar)

In [120]:
# make a raster for the marine only
out_raster= arcpy.sa.Con("island3size", 1, 0, "Value = 1"); out_raster.save(OutMar)

In [122]:
# export vector islands
arcpy.management.CopyFeatures("islands", r"D:\SE_Blueprint_2023\Islands\Islands.shp", '', None, None, None)

<Result 'D:\\SE_Blueprint_2023\\Islands\\Islands.shp'>

### Finalize indiator

do final steps for all indicators to add description fields, clip and export to extent

In [123]:
# set code block for next step

codeblock = """
def Reclass(v):   
	if v == 4:
		return '4 = Large island'
	elif v == 3:
		return '3 = Medium island'
	elif v == 2:
		return '2 = Small island'
	elif v == 1:
		return '1 = Marine'
"""

In [124]:
# add and calculate description field to hold indicator values
arcpy.management.CalculateField("island3size", "descript", "Reclass(!value!)", "PYTHON3", codeblock, "TEXT")

<Result 'D:\\SE_Blueprint_2023\\Islands\\Isl.gdb\\island3size'>

In [125]:
# clip to extent
# with arcpy.EnvManager(outputCoordinateSystem=sr, extent=Rextent, snapRaster=Rextent, cellSize=Rextent):
#    out_raster = arcpy.sa.ExtractByMask("indicator", Rextent); out_raster.save("indicator1")

In [126]:
# export as .tif 
with arcpy.EnvManager(outputCoordinateSystem=sr, extent=Rextent, snapRaster=Rextent, cellSize=Rextent):
    arcpy.management.CopyRaster("island3size", Out2, '', None, "255", "NONE", "NONE", "8_BIT_UNSIGNED", "NONE", "NONE", "TIFF", "NONE", "CURRENT_SLICE", "NO_TRANSPOSE")

In [127]:
# export as .tif 
with arcpy.EnvManager(outputCoordinateSystem=sr, extent=Rextent, snapRaster=Rextent, cellSize=Rextent):
    arcpy.management.CopyRaster("islandsR2", Out, '', None, "255", "NONE", "NONE", "8_BIT_UNSIGNED", "NONE", "NONE", "TIFF", "NONE", "CURRENT_SLICE", "NO_TRANSPOSE")

In [128]:
# this prints the time it took this notebook to run in seconds
end = time.time()
print(end - start)

368.6805348396301
